# description과 리프노드분류로 분류한 정보가 있는 파일 

In [1]:
import pandas as pd
data = pd.read_csv("유틸리티_개요_리프.csv", encoding='cp949')

# 새로운 딕셔너리 생성
new_classification_dict = {}

for _, row in data.iterrows():
    # 각 sub_classfication 값을 쉼표로 분리
    for category in row['sub_classfication'].split(', '):
        companies = new_classification_dict.get(category, [])
        if row['Company'] not in companies:
            companies.append(row['Company'])
        new_classification_dict[category] = companies

new_classification_dict

{'전기 생산': ['대성에너지', '지역난방공사', '한진중공업홀딩스', '한전산업', '한전KPS'],
 '전기 배급': ['대성에너지', '지역난방공사', '한진중공업홀딩스', '한전산업', '한전KPS'],
 '가스 제조': ['대성에너지',
  'SK가스',
  'E1',
  '한진중공업홀딩스',
  '대성홀딩스',
  '인천도시가스',
  '경동도시가스',
  '삼천리',
  '한국가스공사',
  '지에스이'],
 '가스 배급': ['대성에너지',
  'SK가스',
  'E1',
  '한진중공업홀딩스',
  '대성홀딩스',
  '인천도시가스',
  '경동도시가스',
  '삼천리',
  '한국가스공사',
  '지에스이'],
 '에너지 거래': ['대성에너지', 'SK가스', '지역난방공사', '삼천리', '한전산업', '한전KPS', '한국가스공사'],
 '독립 전력 생산': ['지역난방공사', '한전KPS'],
 '유틸리티': ['한전기술'],
 '복합 공익 사업체': ['삼천리']}

# 리프노드_기업들로 csv 파일 저장코드

In [2]:
# 딕셔너리를 데이터프레임으로 변환
classification_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in new_classification_dict.items()]))

# 데이터프레임을 CSV 파일로 저장
classification_csv_path = '유틸리티_리프노드_기업들.csv'
classification_df.to_csv(classification_csv_path, encoding='cp949', index=False)

classification_csv_path

'유틸리티_리프노드_기업들.csv'

# 리프(소)분류 -> 중분류

In [4]:
# CSV 파일을 읽는다
df = pd.read_csv("유틸리티_리프노드_기업들.csv", encoding='cp949')

# DataFrame을 딕셔너리로 변환한다
small_dict = {}
for column in df.columns:
    # NaN 값은 무시한다
    small_dict[column] = df[column].dropna().tolist()

print(small_dict)

{'전기 생산': ['대성에너지', '지역난방공사', '한진중공업홀딩스', '한전산업', '한전KPS'], '전기 배급': ['대성에너지', '지역난방공사', '한진중공업홀딩스', '한전산업', '한전KPS'], '가스 제조': ['대성에너지', 'SK가스', 'E1', '한진중공업홀딩스', '대성홀딩스', '인천도시가스', '경동도시가스', '삼천리', '한국가스공사', '지에스이'], '가스 배급': ['대성에너지', 'SK가스', 'E1', '한진중공업홀딩스', '대성홀딩스', '인천도시가스', '경동도시가스', '삼천리', '한국가스공사', '지에스이'], '에너지 거래': ['대성에너지', 'SK가스', '지역난방공사', '삼천리', '한전산업', '한전KPS', '한국가스공사'], '독립 전력 생산': ['지역난방공사', '한전KPS'], '유틸리티': ['한전기술'], '복합 공익 사업체': ['삼천리']}


In [6]:
mid_class = {
    '전기유틸리티' : ['전기 생산', '전기 배급'],
    '가스유틸리티' : ['가스 제조', '가스 배급'],
    '복합유틸리티' : ['복합 공익 사업체'],
    '독립 전력 생산 및 에너지 거래' : ['독립 전력 생산', '에너지 거래'],
    '유틸리티' : ['유틸리티']
}

In [7]:
# 소분류 -> 중분류
new_classification_companies = {}

for new_category, old_categories in mid_class.items():
    new_classification_companies[new_category] = []
    
    for old_category in old_categories:
        if old_category in small_dict:
            new_classification_companies[new_category].extend(small_dict[old_category])
            
    # 중복된 기업명 제거
    new_classification_companies[new_category] = list(set(new_classification_companies[new_category]))

# 새로운 분류로 DataFrame을 생성하고 CSV로 저장한다
df_new_classified = pd.DataFrame.from_dict(new_classification_companies, orient='index').transpose()
df_new_classified.to_csv('유틸리티_중분류_기업들.csv', index=False, encoding='cp949')

# 중분류 -> 대분류

In [8]:
big_class = {
    '유틸리티' : ['유틸리티', '전기유틸리티', '가스유틸리티', '복합유틸리티', '독립 전력 생산 및 에너지 거래']
}

In [9]:
# CSV 파일을 읽는다
df = pd.read_csv("유틸리티_중분류_기업들.csv", encoding='cp949')

# DataFrame을 딕셔너리로 변환한다
mid_dict = {}
for column in df.columns:
    # NaN 값은 무시한다
    mid_dict[column] = df[column].dropna().tolist()

print(mid_dict)

{'전기유틸리티': ['대성에너지', '한진중공업홀딩스', '한전KPS', '한전산업', '지역난방공사'], '가스유틸리티': ['대성에너지', '경동도시가스', '한진중공업홀딩스', 'SK가스', '한국가스공사', 'E1', '인천도시가스', '삼천리', '대성홀딩스', '지에스이'], '복합유틸리티': ['삼천리'], '독립 전력 생산 및 에너지 거래': ['대성에너지', 'SK가스', '한전KPS', '한국가스공사', '삼천리', '한전산업', '지역난방공사'], '유틸리티': ['한전기술']}


In [10]:
# 중분류 -> 대분류
new_classification_companies = {}

for new_category, old_categories in big_class.items():
    new_classification_companies[new_category] = []
    
    for old_category in old_categories:
        if old_category in mid_dict:
            new_classification_companies[new_category].extend(mid_dict[old_category])
            
    # 중복된 기업명 제거
    new_classification_companies[new_category] = list(set(new_classification_companies[new_category]))

# 새로운 분류로 DataFrame을 생성하고 CSV로 저장한다
df_new_classified = pd.DataFrame.from_dict(new_classification_companies, orient='index').transpose()
df_new_classified.to_csv('유틸리티_대분류_기업들.csv', index=False, encoding='cp949')

In [11]:
# CSV 파일을 읽는다
df = pd.read_csv("유틸리티_대분류_기업들.csv", encoding='cp949')

# DataFrame을 딕셔너리로 변환한다
big_dict = {}
for column in df.columns:
    # NaN 값은 무시한다
    big_dict[column] = df[column].dropna().tolist()

print(big_dict)

{'유틸리티': ['대성에너지', '경동도시가스', '한진중공업홀딩스', 'SK가스', '한전KPS', '한국가스공사', 'E1', '한전기술', '인천도시가스', '삼천리', '대성홀딩스', '한전산업', '지에스이', '지역난방공사']}
